In [21]:
# This code is imported from the following project: https://github.com/asmith26/wide_resnets_keras

import logging
import sys
import numpy as np
from keras.models import Model
from keras.layers import Input, Activation, add, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

sys.setrecursionlimit(2 ** 20)
np.random.seed(2 ** 10)


class WideResNet:
    def __init__(self, image_size, depth=16, k=8):
        self._depth = depth
        self._k = k
        self._dropout_probability = 0
        self._weight_decay = 0.0005
        self._use_bias = False
        self._weight_init = "he_normal"

        if K.image_dim_ordering() == "th":
            logging.debug("image_dim_ordering = 'th'")
            self._channel_axis = 1
            self._input_shape = (3, image_size, image_size)
        else:
            logging.debug("image_dim_ordering = 'tf'")
            self._channel_axis = -1
            self._input_shape = (image_size, image_size, 3)

    # Wide residual network http://arxiv.org/abs/1605.07146
    def _wide_basic(self, n_input_plane, n_output_plane, stride):
        def f(net):
            # format of conv_params:
            #               [ [kernel_size=("kernel width", "kernel height"),
            #               strides="(stride_vertical,stride_horizontal)",
            #               padding="same" or "valid"] ]
            # B(3,3): orignal <<basic>> block
            conv_params = [[3, 3, stride, "same"],
                           [3, 3, (1, 1), "same"]]

            n_bottleneck_plane = n_output_plane

            # Residual block
            for i, v in enumerate(conv_params):
                if i == 0:
                    if n_input_plane != n_output_plane:
                        net = BatchNormalization(axis=self._channel_axis)(net)
                        net = Activation("relu")(net)
                        convs = net
                    else:
                        convs = BatchNormalization(axis=self._channel_axis)(net)
                        convs = Activation("relu")(convs)

                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)
                else:
                    convs = BatchNormalization(axis=self._channel_axis)(convs)
                    convs = Activation("relu")(convs)
                    if self._dropout_probability > 0:
                        convs = Dropout(self._dropout_probability)(convs)
                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)

            # Shortcut Connection: identity function or 1x1 convolutional
            #  (depends on difference between input & output shape - this
            #   corresponds to whether we are using the first block in each
            #   group; see _layer() ).
            if n_input_plane != n_output_plane:
                shortcut = Conv2D(n_output_plane, kernel_size=(1, 1),
                                         strides=stride,
                                         padding="same",
                                         kernel_initializer=self._weight_init,
                                         kernel_regularizer=l2(self._weight_decay),
                                         use_bias=self._use_bias)(net)
            else:
                shortcut = net

            return add([convs, shortcut])

        return f


    # "Stacking Residual Units on the same stage"
    def _layer(self, block, n_input_plane, n_output_plane, count, stride):
        def f(net):
            net = block(n_input_plane, n_output_plane, stride)(net)
            for i in range(2, int(count + 1)):
                net = block(n_output_plane, n_output_plane, stride=(1, 1))(net)
            return net

        return f

#    def create_model(self):
    def __call__(self):
        logging.debug("Creating model...")

        assert ((self._depth - 4) % 6 == 0)
        n = (self._depth - 4) / 6

        inputs = Input(shape=self._input_shape)

        n_stages = [16, 16 * self._k, 32 * self._k, 64 * self._k]

        conv1 = Conv2D(filters=n_stages[0], kernel_size=(3, 3),
                              strides=(1, 1),
                              padding="same",
                              kernel_initializer=self._weight_init,
                              kernel_regularizer=l2(self._weight_decay),
                              use_bias=self._use_bias)(inputs)  # "One conv at the beginning (spatial size: 32x32)"

        # Add wide residual blocks
        block_fn = self._wide_basic
        conv2 = self._layer(block_fn, n_input_plane=n_stages[0], n_output_plane=n_stages[1], count=n, stride=(1, 1))(conv1)
        conv3 = self._layer(block_fn, n_input_plane=n_stages[1], n_output_plane=n_stages[2], count=n, stride=(2, 2))(conv2)
        conv4 = self._layer(block_fn, n_input_plane=n_stages[2], n_output_plane=n_stages[3], count=n, stride=(2, 2))(conv3)
        batch_norm = BatchNormalization(axis=self._channel_axis)(conv4)
        relu = Activation("relu")(batch_norm)

        # Classifier block
        pool = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), padding="same")(relu)
        flatten = Flatten()(pool)
        predictions_g = Dense(units=2, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax")(flatten)
        predictions_a = Dense(units=101, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax")(flatten)

        model = Model(inputs=inputs, outputs=[predictions_g, predictions_a])

        return model

In [22]:
import cv2
import os
from time import sleep
import numpy as np
import argparse
from keras.utils.data_utils import get_file

In [23]:
class Face_Object(object):
    
    # This pre-trained has been taken from https://github.com/Tony607/Keras_age_gender/releases/download/V1.0/weights.18-4.06.hdf5
    haar_cascade_classifier = "haarcascade_frontalface_alt.xml"
    WRN_WEIGHTS_PATH = ".\\pretrained_models\\weights.18-4.06.hdf5"
    
    def __new__(Cls, weights=None, depth=16, width=8, face_size=64):
        if not hasattr(Cls, 'instance'):
            Cls.instance = super(Face_Object, Cls).__new__(Cls)
        return Cls.instance

    def __init__(self, depth=16, width=8, face_size=64):
        self.face_size = face_size
        self.model = WideResNet(face_size, depth=depth, k=width)()
        model_dir = os.path.join(os.getcwd(), "pretrained_models").replace("//", "\\")
        fpath = get_file('weights.18-4.06.hdf5',
                        self.WRN_WEIGHTS_PATH,
                         cache_subdir=model_dir)
        self.model.load_weights(fpath)

    def draw_label (Cls, image, point, label, font=cv2.FONT_HERSHEY_COMPLEX , font_scale=1, thickness=1):
        size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        x, y = point
        cv2.rectangle (image, (x, y - size[1]), (x + size[0], y), (0,255,4), cv2.FILLED)
        cv2.putText (image, label, point, font, font_scale, (255, 255, 255), thickness)

    def crop_face (self, imgarray, section, margin=40, size=64):
        # Storing the height, width and putting down the channel
        img_h, img_w, _ = imgarray.shape
        
        #Check if the face has area
        if section is None:
            section = [0, 0, img_w, img_h]   #Put empty co-ordinates to section if section is none
        
        # Assign the follwing variables 
        (x, y, w, h) = section
        
        #Calculate the margin
        margin = int(min(w,h) * margin / 100)
        
        x_a = x - margin
        y_a = y - margin
        x_b = x + w + margin
        y_b = y + h + margin
        
        if x_a < 0:
            x_b = min(x_b - x_a, img_w-1)
            x_a = 0
        if y_a < 0:
            y_b = min(y_b - y_a, img_h-1)
            y_a = 0
        if x_b > img_w:
            x_a = max(x_a - (x_b - img_w), 0)
            x_b = img_w
        if y_b > img_h:
            y_a = max(y_a - (y_b - img_h), 0)
            y_b = img_h
         
        # Cropping the face out of the whole image
        cropped = imgarray[y_a: y_b, x_a: x_b]
        
        # Resize the image to the 3 x 3
        resized_img = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
        
        # Converting the resized image to a numpy array
        resized_img = np.array(resized_img)
        
        # Returning the resized image with shape (size x size x 3) 
        return resized_img, (x_a, y_a, x_b - x_a, y_b - y_a)

    def find_face (self):
        
        # Get a Face classifier 
        face_cascade = cv2.CascadeClassifier(self.haar_cascade_classifier)
        
        # Open WebCam
        video_capture = cv2.VideoCapture(0)
        
        while True:
            if not video_capture.isOpened():
                sleep(5)
            
            # Capture frame-by-frame
            ret, frame = video_capture.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.2,
                minNeighbors=10,
                minSize=(self.face_size, self.face_size)
            )
            
            # placeholder for cropped faces
            face_imgs = np.empty ((len(faces), self.face_size, self.face_size, 3))
            
            # returns i as an integer and face as a seq[i]
            for i, face in enumerate(faces):
                face_img, cropped = self.crop_face (frame, face, margin=40, size=self.face_size)
                # Taking the parameters of the cropped face
                (x, y, w, h) = cropped
                # Drawing a rectangle around face
                cv2.rectangle (frame, (x, y), (x + w, y + h), (53,196,55), 2) 
                # Converting it into a format that can be accepted by predict ()
                face_imgs [i,:,:,:] = face_img
                    
            if len(face_imgs) > 0:
                # predict ages and genders of the detected faces
                results = self.model.predict (face_imgs) # Putting the placholder for cropped faces 
                predicted_genders = results[0]  
                ages = np.arange(0, 101).reshape(101, 1)
                predicted_ages = results[1].dot(ages).flatten()
                
            # draw results
            for i, face in enumerate(faces):
                label = "{}, {}".format(int(predicted_ages[i]),
                                        "Female" if predicted_genders[i][0] > 0.5 else "Male")
                self.draw_label(frame, (face[0], face[1]), label)
                
            cv2.imshow('Predicted Age And Gender', frame)
            if cv2.waitKey(5) == 27:  # ESC key press
                break
                
        # When everything is done, release the capture
        video_capture.release()
        cv2.destroyAllWindows()

In [24]:
def main():
    face = Face_Object(depth=16, width=8)

    face.find_face ()

In [30]:
if __name__ == "__main__":
    main()